In [ ]:
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.metrics import mean_squared_error
from math import sqrt
import pandas as pd

In [ ]:
with open("../tmp/predictions_with_no_events_collector.pkl", "rb") as f:
    collector = pickle.load(f)

In [ ]:
model_names = list(collector.keys())

In [ ]:
shuffled_list = random.sample(model_names, len(model_names))

In [ ]:
shuffled_list[0]

In [ ]:
model_names

In [ ]:
model_name = "sarimax_all_optimized_adj_events_2_FB_demand_part_1_cell_871f81535ffffff.pkl" 
type(collector[model_name]["fact"])

In [ ]:
type(collector[model_name]["predictions"])

In [ ]:
type(collector[model_name]["predictions_with_no_events"])

In [ ]:
colnames = [f"{el}_{dep_var}" for dep_var in ["demand", "supply"] for el in ["fact", "predictions", "predictions_with_no_events"]]

In [ ]:
colnames

In [ ]:
model_name_DD_1 = "sarimax_all_optimized_adj_events_2_DD_supply_part_2_cell_871f1b54effffff.pkl" 
model_name_DD_2 = "sarimax_all_optimized_adj_events_2_DD_supply_part_1_cell_871f1b54effffff.pkl" 
sr_DD_1 = collector[model_name_DD_1]["fact"]
sr_DD_2 = collector[model_name_DD_2]["fact"]

combinded_sr_DD = pd.concat([sr_DD_1, sr_DD_2])

df_DD = pd.DataFrame(columns=colnames, index = combinded_sr_DD.index)

for colname in  df_DD.columns:
    df_DD[colname] =0 


model_name_FB_1 = "sarimax_all_optimized_adj_events_2_FB_supply_part_1_cell_871f81c99ffffff.pkl" 
model_name_FB_2 = "sarimax_all_optimized_adj_events_2_FB_supply_part_2_cell_871f81c99ffffff.pkl" 
sr_FB_1 = collector[model_name_FB_1]["fact"]
sr_FB_2 = collector[model_name_FB_2]["fact"]

combinded_sr_FB = pd.concat([sr_FB_1, sr_FB_2])

df_FB = pd.DataFrame(columns=colnames, index = combinded_sr_FB.index)

for colname in  df_FB.columns:
    df_FB[colname] =0 





In [ ]:
df_helper = {"DD": df_DD, "FB": df_FB}

In [ ]:
for city in ["DD", "FB"]:
    for dep_var in ["demand", "supply"]:
        for model_name in model_names:
            if city in model_name and dep_var in model_name:
                for el in ["fact", "predictions", "predictions_with_no_events"]:
                    tmp_index = collector[model_name][el].index
                    df_helper[city].loc[tmp_index, f"{el}_{dep_var}"] = df_helper[city].loc[tmp_index, f"{el}_{dep_var}"]+ collector[model_name][el]

                

In [ ]:
df_DD

In [ ]:
df_FB

In [ ]:
df_DD["predictions_demand"].sum()

In [ ]:
df_DD = df_DD.reset_index()

In [ ]:
df_FB = df_FB.reset_index()

In [ ]:
def plot_tmp(tmp_input):
    tmp_input_copy = tmp_input.copy()
    for dep_var in ["demand", "supply"]:
        plt.figure(figsize=(8, 5))  # 10, 5 was too wide
        sns.lineplot(data=tmp_input_copy[f"fact_{dep_var}"], label="Test data")
        sns.lineplot(data=tmp_input_copy[f"predictions_{dep_var}"], label="Predictions", linestyle="--")
        sns.lineplot(data=tmp_input_copy[f"predictions_with_no_events_{dep_var}"], label="Predictions with 0 events", linestyle="--")
        plt.xlabel("Datetime hour")
        plt.xticks(rotation=90)
        plt.legend()
        plt.tight_layout()
        plt.show()


In [ ]:
df_tmp = df_DD.loc[df_DD.datetime_hour<=pd.to_datetime("2024-08-01")]
print("df_DD part 1")
plot_tmp(df_tmp)


df_tmp = df_DD.loc[df_DD.datetime_hour>=pd.to_datetime("2024-08-01")]
print("df_DD part 2")
plot_tmp(df_tmp)


df_tmp = df_FB.loc[df_FB.datetime_hour<=pd.to_datetime("2024-08-01")]
print("df_FB part 1")
plot_tmp(df_tmp)

df_tmp = df_FB.loc[df_FB.datetime_hour>=pd.to_datetime("2024-08-01")]
print("df_FB part 2")
plot_tmp(df_tmp)



In [ ]:
(df_DD["predictions_demand"].sum()-df_DD["predictions_with_no_events_demand"].sum())/df_DD["predictions_demand"].sum()*100

In [ ]:
(df_DD["predictions_supply"].sum()-df_DD["predictions_with_no_events_supply"].sum())/df_DD["predictions_supply"].sum()*100

In [ ]:
(df_FB["predictions_demand"].sum()-df_FB["predictions_with_no_events_demand"].sum())/df_FB["predictions_demand"].sum()*100

In [ ]:
(df_FB["predictions_supply"].sum()-df_FB["predictions_with_no_events_supply"].sum())/df_FB["predictions_supply"].sum()*100

In [ ]:
df_DD.columns

In [ ]:
df_DD["predictions_demand"].sum()

In [ ]:
# for model_name in model_names:
#     fact = collector[model_name]["fact"]
#     predictions =  collector[model_name]["predictions"]
#     predictions_with_no_events =  collector[model_name]["predictions_with_no_events"]

#     rmse = sqrt(mean_squared_error(predictions, predictions_with_no_events))
#     if rmse > 0.5:
#         plt.figure(figsize=(8, 5))  # 10, 5 was too wide
#         sns.lineplot(data=fact, label="Test data")
#         sns.lineplot(data=predictions, label="Predictions", linestyle="--")
#         sns.lineplot(data=predictions_with_no_events, label="Predictions with 0 events", linestyle="--")
#         plt.xlabel("Datetime hour")
#         plt.xticks(rotation=90)
#         plt.legend()
#         plt.tight_layout()
#         plt.show()